Titanic with Keras

In [2]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten, Dense, Activation, Dropout

Setup

In [3]:
DATA_HOME_DIR = "/user/rh186032/datasciencecoursera/TitanicML/input/"

raw_data = pd.read_csv(DATA_HOME_DIR + 'train.csv', index_col=0)
test_data = pd.read_csv(DATA_HOME_DIR + 'test.csv', index_col=0)

In [4]:
test_ind = test_data.index

data= raw_data[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare']].dropna()
t_data = test_data[['Pclass','Sex','Age','SibSp','Parch','Fare']]

In [5]:
Pclass = pd.get_dummies(data['Pclass'])
Pclass.columns=['1st','2nd','3rd']
Pclass = Pclass.drop('1st',axis=1)

tPclass = pd.get_dummies(t_data['Pclass'])
tPclass.columns=['1st','2nd','3rd']
tPclass = tPclass.drop('1st',axis=1)

Sex = pd.get_dummies(data['Sex'])
Sex = Sex.drop('male',axis=1)

tSex = pd.get_dummies(t_data['Sex'])
tSex = tSex.drop('male',axis=1)

def adult_kids(age):
    if age > 15:
        if age > 60:
            return  'older'
        else :
            return   'adult'
    else :
        if age < 16:
            return   'kids'
        else :
            return np.nan

data['adult_kids'] = data['Age'].apply(adult_kids)
Age_cat = pd.get_dummies(data['adult_kids']).drop('adult',axis=1)
t_data['adult_kids'] = t_data['Age'].apply(adult_kids)
tAge_cat = pd.get_dummies(t_data['adult_kids']).drop('adult',axis=1)

Data_tmp = data[['Survived','SibSp','Parch','Fare']]

Merge_data = pd.merge(Data_tmp,Pclass,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,Sex,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,Age_cat,right_index=True,left_index=True)
y = Merge_data['Survived'].values
x = Merge_data.drop('Survived',axis=1).values

Data_tmp = t_data[['SibSp','Parch','Fare']]

Merge_data = pd.merge(Data_tmp,tPclass,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,tSex,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,tAge_cat,right_index=True,left_index=True)
tx = Merge_data

(x.shape, y.shape, tx.shape)

/usr/lib/python3.4/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


((714, 8), (714,), (418, 8))

Build a model

In [6]:
# create model
model = Sequential()
model.add(Dense(64, input_shape=(8,)))

for i in range(0, 8):
    model.add(Dense(units=64))
    model.add(Activation('relu'))

model.add(Dense(units=1))
model.add(Activation('linear'))

model.compile(loss='mean_squared_error', optimizer='rmsprop')

In [9]:
model.fit(x, y, epochs=64)

Epoch 1/64
714/714 [==============================] - 0s - loss: 0.2889     
Epoch 2/64
714/714 [==============================] - 0s - loss: 0.2774     
Epoch 3/64
714/714 [==============================] - 0s - loss: 0.2544     
Epoch 4/64
714/714 [==============================] - 0s - loss: 0.2459     
Epoch 5/64
714/714 [==============================] - 0s - loss: 0.2579     

In [10]:
p_survived = model.predict_classes(tx.values)

 32/418 [=>............................] - ETA: 0s

In [11]:
submission = pd.DataFrame()
submission['PassengerId'] = test_ind
submission['Survived'] = p_survived

In [12]:
submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,1
8,900,1
9,901,1


In [13]:
submission.to_csv('submission.csv', index=False)